### Import dependencies

In [1]:
import pandas as pd
import numpy as np

### Tokenizing results

In [49]:
classifier_results = pd.read_csv("data/results/classifiers-results.csv")
classifier_results.head()

,target,RFC,MLP,SVC,LinearSVC,NuSVC,SGD,LR,KNC,LSTM,BERT
0,advocate,advocate,advocate,advocate,advocate,advocate,advocate,advocate,advocate,advocate,advocate
1,pilot,pilot,pilot,pilot,pilot,pilot,pilot,pilot,pilot,pilot,pilot
2,machine learn engineer,machine learn engineer,machine learn engineer,machine learn engineer,machine learn engineer,machine learn engineer,machine learn engineer,machine learn engineer,machine learn engineer,machine learn engineer,machine learn engineer
3,data scientist,data scientist,data scientist,data scientist,data scientist,data scientist,data scientist,data scientist,data scientist,data scientist,data scientist
4,construction manager,construction manager,construction manager,construction manager,construction manager,construction manager,construction manager,construction manager,construction manager,construction manager,construction manager


In [50]:
from sklearn.preprocessing import LabelEncoder

def encode_labels(data):
    label_encoder = LabelEncoder()
    label_encoder.fit(data)
    return label_encoder

def decoded_label(label_encoder, y_encoded, categorical=True):
    label = [int(np.argmax(y)) for y in y_encoded] if categorical else y_encoded
    return label_encoder.inverse_transform(label)

In [51]:
all_values = []
for col in classifier_results.columns:
    for v in classifier_results[col].values:
        all_values.append(v)

all_values = pd.DataFrame(all_values, columns=['values'])
label_encoder = encode_labels(all_values['values'])

for col in classifier_results.columns:
    classifier_results[col] = label_encoder.transform(classifier_results[col])

classifier_results.head()

,target,RFC,MLP,SVC,LinearSVC,NuSVC,SGD,LR,KNC,LSTM,BERT
0,1,1,1,1,1,1,1,1,1,1,1
1,71,71,71,71,71,71,71,71,71,71,71
2,61,61,61,61,61,61,61,61,61,61,61
3,31,31,31,31,31,31,31,31,31,31,31
4,23,23,23,23,23,23,23,23,23,23,23


In [77]:
from sklearn.model_selection import train_test_split

X = classifier_results.drop('target', axis=1)
y = classifier_results['target']
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1915, 10), (821, 10), (1915,), (821,))

### Dynamic choise

In [85]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error as mse

knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(X_train)
dynamic_predictions = []
for t in X_test:
    distances, indices = knn.kneighbors(t.reshape(1, -1))
    y_kn = y[np.ravel(indices)]
    x_kn = X.iloc[np.ravel(indices)]

    weight = np.linalg.pinv(x_kn).dot(y_kn)
    dynamic_predictions.append(t.dot(weight))

dynamic_error = mse(y_test, dynamic_predictions)
dynamic_error

43.2322260743865